In [1]:
# from huggingface_hub import login
# login() # Get your token here

In [1]:
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from utils.extract import get_score, clean_output, get_essay
from utils.prompt import rescore, indepth_feedback, until_correct, get_instruction_prompt, get_system_prompt

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "mistralai/Mistral-7B-v0.1" # Super loose
# model_id = 'google/gemma-7b-it'
# model_id = "microsoft/Phi-3-mini-4k-instruct"
mode = 'easy'
if 'llama' in model_id:
    mode = 'harsh'
    
chatbot_role = 'system'
change_mode = True
# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model_id,
#     model_kwargs={"torch_dtype": torch.bfloat16},
#     device_map="auto",
# )

model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained(model_id)


2024-04-29 22:14:29.911104: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 22:14:29.911137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 22:14:29.912097: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-29 22:14:30.481826: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
EXAMPLES = 'cambridge/1x_3_7'

**Cam 17**

- Test 1: 6.5 7
- Test 2: 6.5 6
- Test 3: 6.5 7
- Test 4: 6 6.5

In [3]:
    
with open('prompt/output_suggestion_format.txt', 'r') as f:
    output_suggestion = f.read()
    
with open(f'sample/{EXAMPLES}/question.txt', 'r') as f:
    essay_topic = f.read()

with open(f'sample/{EXAMPLES}/answer.txt', 'r') as f:
    student_response = f.read()


## Get the System prompt

In [4]:
system_prompt = get_system_prompt()
instruction_prompt_1 = get_instruction_prompt(essay_topic, student_response)

## First generate result

In [5]:
messages = [
    # {"role": "user", "content": system_prompt},
    # {"role": chatbot_role, "content": "Sure"},
    {"role": chatbot_role, "content": system_prompt},
    {"role": "user", "content": instruction_prompt_1},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)
model_args = {
    "max_new_tokens": 2000,
    "return_full_text": False,
    "temperature": 0.6,
    "top_p":0.9,
    "do_sample": True,
}

generation_args = {
    "model_args": model_args,
    "role": chatbot_role,
}


output1 = pipe(messages, **generation_args["model_args"])
output_1_text = output1[0]["generated_text"]
messages.append({"role": chatbot_role, "content": output_1_text})
print(output_1_text)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


**General: 6.5**
Overall Explanation: The essay provides a good overview of the topic, discussing both sides of the argument and presenting a clear opinion. However, the essay lacks depth and analysis, with some points being made without sufficient evidence or examples. The writing is clear, but the vocabulary is limited, and there are some errors in grammar and punctuation.

**Task Response: 6**
Explanation: The essay addresses the topic, but the discussion is limited and lacks depth. The writer presents both sides of the argument, but the points made are not fully developed, and some are not supported with evidence. The writer's own opinion is clear, but it is not fully developed or supported with examples.

**Coherence and Cohesion: 6**
Explanation: The essay is generally easy to follow, with a clear introduction, body, and conclusion. The writer uses transitional phrases to connect ideas, but the linking between paragraphs is not always smooth. There are some instances of repetitio

In [6]:
adjust_output  = until_correct(pipe, messages, **generation_args)
general, tr, cc, lr, gr = get_score(adjust_output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


______Adjusting score 1_______
Inconsistent scoring


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


______Adjusting score 2_______
Inconsistent scoring


In [7]:
print(clean_output(adjust_output))

**General: 5.5**
Overall Explanation: The essay provides a good overview of the topic, discussing both sides of the argument and presenting a clear opinion. However, the essay lacks depth and analysis, with some points being made without sufficient evidence or examples. The writing is clear, but the vocabulary is limited, and there are some errors in grammar and punctuation.

**Task Response: 6**
Explanation: The essay addresses the topic, but the discussion is limited and lacks depth. The writer presents both sides of the argument, but the points made are not fully developed, and some are not supported with evidence. The writer's own opinion is clear, but it is not fully developed or supported with examples.

**Coherence and Cohesion: 6**
Explanation: The essay is generally easy to follow, with a clear introduction, body, and conclusion. The writer uses transitional phrases to connect ideas, but the linking between paragraphs is not always smooth. There are some instances of repetitio

In [8]:
messages = [
    # {"role": "user", "content": system_prompt},
    # {"role": chatbot_role, "content": "Sure, I will be a scoring assistant for your IELTS writing task 2 essay."},
    {"role": generation_args["role"], "content": system_prompt},
    {"role": "user", "content": instruction_prompt_1},
    {"role": generation_args["role"], "content": clean_output(adjust_output)},
]

In [9]:
if change_mode:
    rescore_output = rescore(pipe, messages, mode, **generation_args)
    rescore_output = until_correct(pipe, messages, **generation_args)
    general, tr, cc, lr, gr = get_score(rescore_output)
    print(rescore_output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


I apologize for the harsh evaluation. Upon re-evaluation, I will provide a more accurate and fair assessment of the essay.

**General: 6.5**
Overall Explanation: The essay provides a good overview of the topic, discussing both sides of the argument and presenting a clear opinion. While the writing is clear, the vocabulary is limited, and there are some errors in grammar and punctuation. The essay could benefit from more depth and analysis, but it is generally well-structured and easy to follow.

**Task Response: 7**
Explanation: The essay addresses the topic adequately, presenting both sides of the argument and the writer's own opinion. The discussion is clear, but some points are made without sufficient evidence or examples. The writer could benefit from providing more supporting details and examples to strengthen their argument.

**Coherence and Cohesion: 7**
Explanation: The essay is generally easy to follow, with a clear introduction, body, and conclusion. The writer uses transitio

### Provide in-depth feedback

In [10]:
first_output = clean_output(adjust_output)
if change_mode:
    first_output = clean_output(rescore_output)
messages = [
    {"role": generation_args["role"], "content": system_prompt},
    {"role": "user", "content": instruction_prompt_1},
    {"role": generation_args["role"], "content": first_output},
]
rcm_output = indepth_feedback(pipe, messages, **generation_args)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [11]:
print(rcm_output)

**General: 6.5**
Overall Explanation: The essay provides a good overview of the topic, discussing both sides of the argument and presenting a clear opinion. While the writing is clear, the vocabulary is limited, and there are some errors in grammar and punctuation. The essay could benefit from more depth and analysis, but it is generally well-structured and easy to follow.

**Task Response: 7**
Explanation: The essay addresses the topic adequately, presenting both sides of the argument and the writer's own opinion. However, the discussion is somewhat superficial, and the writer could benefit from providing more supporting details and examples to strengthen their argument. For instance, the writer mentions that advertisements can be successful in persuading people to buy things, but they do not provide any specific examples or statistics to support this claim. Additionally, the writer's own opinion is not fully developed, and they could benefit from providing more evidence and analysis 

## Making adjustment to the essay

In [12]:
MAX_TARGET = 7.5
ADJUSTMENT_MODE = 'close_to_original'

In [13]:
if ADJUSTMENT_MODE =='close_to_original' and change_mode:
    general, tr, cc, lr, gr = get_score(adjust_output)
    first_output = clean_output(adjust_output)
    messages = [
        {"role": generation_args["role"], "content": system_prompt},
        {"role": "user", "content": instruction_prompt_1},
        {"role": generation_args["role"], "content": first_output},
    ]
    indepth_feedback(pipe, messages, **generation_args)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [14]:
CURRENT_TARGET = general

if CURRENT_TARGET <= MAX_TARGET - 0.5:
    messages.append({"role": "user", "content": f"Making adjustment directly into the essay to improve at least 1 score in each metric. {output_suggestion}"})
    suggest_essay = pipe(messages, **generation_args["model_args"])
    
else: 
    messages.append({"role": "user", "content": f"Making adjustment directly into the essay to optimize the essay. {output_suggestion}"})
    suggest_essay = pipe(messages, **generation_args["model_args"])


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [15]:
print(suggest_essay[0]['generated_text'])

**Suggesting changes:**

**Task Response:**

* Provide more specific examples and evidence to support the claims made in the essay.

**Coherence and Cohesion:**

* Use transitional phrases to connect ideas between paragraphs, such as "However," "In addition," or "Furthermore."
* Use more varied sentence structures to make the writing more engaging.

**Lexical Resource:**

* Use more nuanced and varied vocabulary to describe the effects of advertisements, such as "subconscious influence" or "mental persuasion."

**Grammatical Range and Accuracy:**

* Use more complex sentence structures, such as compound or complex sentences, to add variety to the writing.
* Check for errors in grammar and punctuation, such as missing articles or incorrect verb tenses.

**Revised Essay:**

Advertisement has always been a crucial part of the world of marketing. Throughout the decades, we have seen a significant increase in the amount of advertisements, whether it is on the media like television or widesp

In [16]:
print(get_essay(suggest_essay[0]['generated_text']))



Advertisement has always been a crucial part of the world of marketing. Throughout the decades, we have seen a significant increase in the amount of advertisements, whether it is on the media like television or widespread through social network platforms. The goal of advertisements is to get consumers to buy a targeted product, and while this method has been proven considerably successful generally, some people view it as too prevalent to catch the consumers' attention any more.

Advertisements can act as a strong persuasion device to seemingly hypnotize people into buying goods and services. This is so because of the tactics placed in the messages, such as showing people having a good time together when using a particular product, using bandwagon, showing only the upsides of usage, and applying compare and contrast strategies to show the effects of using the product and make it stand out. Even if people do not know it, these messages are repeated several times and soon it may brainw